In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep

import pandas as pd
import re
from datetime import datetime
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url = 'https://incamp.ru/camps/education/2/'
response = requests.get(url, headers=headers)
response.text
soup = BeautifulSoup(response.text, 'lxml')
data = soup.find_all('div', class_='camp-item item')

In [ ]:
import re

camps_list = []

for part in data:
    # Название лагеря
    name_tag = part.find('div', class_='float-left title').find('a')
    name = name_tag.text.strip() if name_tag else None

    # Ссылка на лагерь
    link = 'https://incamp.ru' + name_tag['href'] if name_tag else None

    # Город и место
    location_div = part.find('div', class_='float-left title').find('div')
    if location_div:
        location_str = str(location_div)
        clean_str = re.sub(r'<[^>]*>', '', location_str)  # Удаляем все HTML-теги
        clean_str = clean_str.replace('Россия, ', '').replace('РФ, ', '').strip()
        parts = clean_str.split(maxsplit=1)
        city = parts[0] if parts else None
        place = parts[1] if len(parts) > 1 else None
    else:
        city, place = None, None

    # Список тегов
    tags_container = part.find('div', class_='keywords clearfix')
    list_tags = [a.text.strip() for a in tags_container.find_all('a')] if tags_container else []

    # Рейтинг
    rating_block = part.find('div', class_='float-right rating')
    rating = None
    if rating_block:
        rating_span = rating_block.find('span', class_='rating_score')
        if rating_span:
            rating_text = rating_span.text.replace(',', '.').split()[0]
            try:
                rating = float(rating_text)
            except ValueError:
                pass

    # Собираем данные
    camps_list.append({
        'name': name,
        'link': link,
        'city': city,
        'place': place,
        'list_tags': list_tags,
        'rating': rating
    })

# Создаем DataFrame
df = pd.DataFrame(camps_list)

In [ ]:
df

,name,link,city,place,list_tags,rating
0,СОЛКД Чкаловец,https://incamp.ru/camp/gornyiy-orlenok-8979/,Новосибирская,"область, Бурмистрово Санаторно-оздоровительный...","[в реестре, Туроператор, Кешбэк от incamp, Озд...",4.2
1,ГЕРОИ,https://incamp.ru/camp/letniy-trenirovochnyiy-...,Калужская,"область, ОбнинскЭтноотель «Центральная Азия». ...","[Детский тур, Кешбэк от incamp, Спортивный, Те...",4.6
2,VIL Camp,https://incamp.ru/camp/vil-camp-13784/,Владимирская,"область, КиржачПарк-отель ""Сосновый бор""","[Детский тур, Кешбэк от incamp, Языковой, Тема...",5.0
3,Komandor Camp. Научный лагерь,https://incamp.ru/camp/snoubord-i-gornyie-lyiz...,Калужская,"область, Таруса АНО ЦДР «Командор»","[в реестре, Мы здесь были, Кешбэк от incamp, С...",4.4
4,Nordic School,https://incamp.ru/camp/nordic-school-12693/,Москва,и Московская обл.Hilton Garden Inn,"[Детский тур, Мы здесь были, Кешбэк от incamp,...",5.0
5,Настоящий лагерь,https://incamp.ru/camp/nastoyashchiy-lager-12328/,Пермский,"край, Пермь ДОЛ «Восток» (ИП Суворов Александ...","[в реестре, Кешбэк от incamp, Тематический, Тв...",4.2
6,JP Корея Сеул,https://incamp.ru/camp/jp-koreya-seul-13875/,Южная,"Корея, Сеулкампус университета ""Seokyeong""","[Зарубежный отдых, Кешбэк от incamp, Языковой,...",5.0
7,Языковое погружение и профориентация с Абитури...,https://incamp.ru/camp/kanikulyi-v-dubae-2024-...,"ОАЭ,","Дубай""Academic City Dubai""","[Зарубежный отдых, Кешбэк от incamp, Лагерь/ту...",4.7
8,Городской Клуб IQ007,https://incamp.ru/camp/gorodskoy-klub-iq007-13...,г.,"Москва, ул. Рокотова 10к2, Юго-Западный округ,...","[Городской клуб, Кешбэк от incamp, Языковой, Т...",4.8
9,Лига Роботов,https://incamp.ru/camp/park-budushchih-profess...,"Санкт-Петербург,","Комендантский пр-т, 9, Приморский район, м. Ко...","[Городской клуб, Мы здесь были, Кешбэк от inca...",5.0
